- Update: 2023.04.11

# 0. 配置参数

- `rockauto`：ROCKAUTO.COM
- `manufacturer`：Manufacturer
- `part_type`：Part Type
- `serial`：可能是Rockauto的内部编号？

In [1]:
from gevent import monkey
monkey.patch_all()
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions, Chrome
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# = = = = = = = = = = = = = = = = = =

rockauto = 'https://www.rockauto.com/en/parts/'

manufacturer = 'dorman'
part_type = 'parking+brake+cable'
serial = '1696'

chromedriver = '../../chromedriver.exe'

# = = = = = = = = = = = = = = = = = =

chrome_options = ChromeOptions()
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation']) # Chrome正受到自动测试软件的控制
chrome_options.add_argument('--disable-blink-features=AutomationControlled') # window.navigator.webdriver=false
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36')
# chrome_options.add_argument('--proxy-server=http://a002.kdltps.com:15818')
chrome_options.add_argument('blink-settings=imagesEnabled=false')

# = = = = = = = = = = = = = = = = = =

df_menu = pd.read_excel('./menu-' + manufacturer + '_' + part_type + '_' + serial + '.xlsx')
list_part_number = df_menu['Part Number'].to_list()
print('batch：' + str(batch) + ' - 总数量：' + str(len(list_part_number)))
print()

# = = = = = = = = = = = = = = = = = =

df = pd.DataFrame(columns=['Part Number',
                           '标题',
                           '图片',
                           'OEM Number',
                           'Alternate Inventory Number',
                           '车型',
                           '链接',
                           '图片链接',
                           '配件-序号',
                           '配件-标题',
                           '配件-图片',
                           '配件-链接',
                           '配件-品类',
                           '配件-数量',
                           '配件-OEM Number',
                           '配件-Alternate Inventory Number',
                           '配件-图片链接'])

# = = = = = = = = = = = = = = = = = =

work = Queue()
for part_number in list_part_number:
    work.put_nowait(part_number)

# = = = = = = = = = = = = = = = = = =

def crawler():
    global df
    
    while not work.empty():
        part_number = work.get_nowait()
        url = rockauto + manufacturer + ',' + part_number + ',' + part_type + ',' + serial
        driver = Chrome(service=Service(chromedriver), options=chrome_options)
        time.sleep(0.3)
        
        for i in range(5):
            try:
                driver.get(url)

                # = = = = = = = = = = = = = = = = = =

                WebDriverWait(driver, 60, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@title="Buyer\'s Guide"]')))
                buyers_guide = driver.find_element(By.XPATH, '//*[@title="Buyer\'s Guide"]')
                buyers_guide.click()

                WebDriverWait(driver, 60, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@class="buyersguide-nested"]')))
                buyers_guide = driver.find_element(By.XPATH, '//*[@class="buyersguide-nested"]').text.replace('\nPlease refer to catalog for application details.', '')

                # = = = = = = = = = = = = = = = = = =

                info = driver.find_element(By.XPATH, '//*[@class="ra-btn ra-btn-moreinfo"]').get_attribute('href')
                driver.get(info)

                # = = = = = = = = = = = = = = = = = =

                WebDriverWait(driver, 60, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@class="listing-text-row-moreinfo-truck"]')))
                title = driver.find_element(By.XPATH, '//*[@class="listing-text-row-moreinfo-truck"]').text

                # = = = = = = = = = = = = = = = = = =

                dict_alternate = {'alternate_inventory_number': '', 'oem_number': ''}
                WebDriverWait(driver, 60, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@aria-label="Alternate/OEM Part Number(s)"]')))
                list_alternate = [j.text for j in driver.find_elements(By.XPATH, '//*[@aria-label="Alternate/OEM Part Number(s)"]')]
                for j in range(len(list_alternate)):
                    if '{Alternate Inventory Numbers: ' in list_alternate[j]:
                        dict_alternate['alternate_inventory_number'] = list_alternate[j].replace('{Alternate Inventory Numbers: ', '').replace('}', '').replace(', ', ';')
                    elif 'OEM / Interchange Numbers: ' in list_alternate[j]:
                        dict_alternate['oem_number'] = list_alternate[j].replace('OEM / Interchange Numbers: ', '').replace(', ', ';')

                # = = = = = = = = = = = = = = = = = =

                WebDriverWait(driver, 60, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@class="listing-inline-image listing-inline-image-moreinfo"]')))
                pic = driver.find_element(By.XPATH, '//*[@class="listing-inline-image listing-inline-image-moreinfo"]').get_attribute('src')

                # = = = = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'Part Number': [str(part_number)],
                                        '标题': [str(title)],
                                        '图片': [''],
                                        'OEM Number': [str(dict_alternate['oem_number'])],
                                        'Alternate Inventory Number': [str(dict_alternate['alternate_inventory_number'])],
                                        '车型': [str(buyers_guide)],
                                        '链接': [str(url)],
                                        '图片链接': [str(pic)],
                                        '配件-序号': [''],
                                        '配件-标题': [''],
                                        '配件-图片': [''],
                                        '配件-链接': [''],
                                        '配件-品类': [''],
                                        '配件-数量': [''],
                                        '配件-OEM Number': [''],
                                        '配件-Alternate Inventory Number': [''],
                                        '配件-图片链接': ['']})

                # = = = = = = = = = = = = = = = = = =

                list_component = driver.find_elements(By.XPATH, '//*[@aria-label="Components that come in this kit"]/table/tbody/tr/td/table/tbody')[1:]
                if len(list_component) > 0:
                    list_dict_component = []
                    for j in range(len(list_component)):
                        list_dict_component.append({'no': str(j+1),
                                                    'title': list_component[j].find_element(By.XPATH, './tr/td[3]').text.replace(' Info', ''),
                                                    'url': list_component[j].find_element(By.XPATH, './tr/td[3]/div[1]/a').get_attribute('href'),
                                                    'category': list_component[j].find_element(By.XPATH, './tr/td[2]').text,
                                                    'quantity': list_component[j].find_element(By.XPATH, './tr/td[1]').text,
                                                    'oem_number': '',
                                                    'alternate_inventory_number': '',
                                                    'pic': ''})

                    # = = = = = = = = = = = = = = = = = =

                    for j in range(len(list_dict_component)):
                        driver.get(list_dict_component[j]['url'])
                        WebDriverWait(driver, 60, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@aria-label="Alternate/OEM Part Number(s)"]')))
                        list_alternate = [k.text for k in driver.find_elements(By.XPATH, '//*[@aria-label="Alternate/OEM Part Number(s)"]')]
                        for k in range(len(list_alternate)):
                            if '{Alternate Inventory Numbers: ' in list_alternate[k]:
                                list_dict_component[j]['alternate_inventory_number'] = list_alternate[k].replace('{Alternate Inventory Numbers: ', '').replace('}', '').replace(', ', ';')
                            elif 'OEM / Interchange Numbers: ' in list_alternate[k]:
                                list_dict_component[j]['oem_number'] = list_alternate[k].replace('OEM / Interchange Numbers: ', '').replace(', ', ';')

                        WebDriverWait(driver, 60, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@class="listing-inline-image listing-inline-image-moreinfo"]')))
                        list_dict_component[j]['pic'] = driver.find_element(By.XPATH, '//*[@class="listing-inline-image listing-inline-image-moreinfo"]').get_attribute('src')

                    # = = = = = = = = = = = = = = = = = =

                    for j in range(len(list_dict_component)):
                        df_temp.loc[j, '配件-序号'] = str(list_dict_component[j]['no'])
                        df_temp.loc[j, '配件-标题'] = str(list_dict_component[j]['title'])
                        df_temp.loc[j, '配件-链接'] = str(list_dict_component[j]['url'])
                        df_temp.loc[j, '配件-品类'] = str(list_dict_component[j]['category'])
                        df_temp.loc[j, '配件-数量'] = str(list_dict_component[j]['quantity'])
                        df_temp.loc[j, '配件-OEM Number'] = str(list_dict_component[j]['oem_number'])
                        df_temp.loc[j, '配件-Alternate Inventory Number'] = str(list_dict_component[j]['alternate_inventory_number'])
                        df_temp.loc[j, '配件-图片链接'] = str(list_dict_component[j]['pic'])
                
                break

            except:
                df_temp = pd.DataFrame({'Part Number': [''],
                                        '标题': [''],
                                        '图片': [''],
                                        'OEM Number': [''],
                                        'Alternate Inventory Number': [''],
                                        '车型': [''],
                                        '链接': [str(url)],
                                        '图片链接': [''],
                                        '配件-序号': [''],
                                        '配件-标题': [],
                                        '配件-图片': [''],
                                        '配件-链接': [''],
                                        '配件-品类': [''],
                                        '配件-数量': [''],
                                        '配件-OEM Number': [''],
                                        '配件-Alternate Inventory Number': [''],
                                        '配件-图片链接': ['']})
                continue

        # = = = = = = = = = = = = = = = = = =

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')
        
        print(part_number, ' [' + str(i) + '] - 剩余数量：' + str(work.qsize()))
        driver.close()
        
# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(7):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df.to_excel('./crawler.xlsx', index=False)
print()
print('搞定')

batch：2 - 总数量：100

C660215  [0] - 剩余数量：93
C660214  [0] - 剩余数量：92
C660218  [0] - 剩余数量：91
C660217  [0] - 剩余数量：90
C660219  [0] - 剩余数量：89
C660220  [0] - 剩余数量：88
C660216  [0] - 剩余数量：87
C660223  [0] - 剩余数量：86
C660224  [0] - 剩余数量：85
C660222  [0] - 剩余数量：84
C660226  [0] - 剩余数量：83
C660225  [0] - 剩余数量：83
C660227  [0] - 剩余数量：81
C660221  [0] - 剩余数量：80
C660228  [0] - 剩余数量：79
C660229  [0] - 剩余数量：78
C660232  [0] - 剩余数量：77
C660231  [0] - 剩余数量：76
C660233  [0] - 剩余数量：75
C660234  [0] - 剩余数量：74
C660230  [0] - 剩余数量：73
C660235  [0] - 剩余数量：72
C660236  [0] - 剩余数量：71
C660237  [0] - 剩余数量：70
C660239  [0] - 剩余数量：69
C660240  [0] - 剩余数量：68
C660241  [0] - 剩余数量：67
C660242  [0] - 剩余数量：66
C660243  [0] - 剩余数量：65
C660244  [0] - 剩余数量：64
C660245  [0] - 剩余数量：63
C660246  [0] - 剩余数量：62
C660248  [0] - 剩余数量：61
C660247  [0] - 剩余数量：60
C660249  [0] - 剩余数量：59
C660250  [0] - 剩余数量：58
C660251  [0] - 剩余数量：57
C660252  [0] - 剩余数量：56
C660253  [0] - 剩余数量：55
C660254  [0] - 剩余数量：54
C660255  [0] - 剩余数量：53
C660256  [0] - 剩余数量：52
C660259  [0] - 

D:\Lennon\python\venv\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
